In [1]:
from collections import OrderedDict
import torch
from torch import nn
from torch.nn import functional as F


class _SimpleSegmentationModel(nn.Module):
    def __init__(self, backbone, classifier, aux_classifier=None):
        super(_SimpleSegmentationModel, self).__init__()
        self.backbone = backbone
        self.classifier = classifier
        self.aux_classifier = aux_classifier

    def forward(self, x):
        input_shape = x.shape[-2:]
        # contract: features is a dict of tensors
        features = self.backbone(x)

        result = OrderedDict()
        x = features["out"]
        
        x = self.classifier(x)
        x = F.interpolate(x, size=input_shape, mode='bilinear', align_corners=False)
        result["out"] = x

        if self.aux_classifier is not None:
            x = features["aux"]
            x = self.aux_classifier(x)
            x = F.interpolate(x, size=input_shape, mode='bilinear', align_corners=False)
            result["aux"] = x

        return result

In [2]:
import torch
from torch import nn
from torch.nn import functional as F



__all__ = ["DeepLabV3"]


class DeepLabV3(_SimpleSegmentationModel):
    """
    Implements DeepLabV3 model from
    `"Rethinking Atrous Convolution for Semantic Image Segmentation"
    <https://arxiv.org/abs/1706.05587>`_.
    Arguments:
        backbone (nn.Module): the network used to compute the features for the model.
            The backbone should return an OrderedDict[Tensor], with the key being
            "out" for the last feature map used, and "aux" if an auxiliary classifier
            is used.
        classifier (nn.Module): module that takes the "out" element returned from
            the backbone and returns a dense prediction.
        aux_classifier (nn.Module, optional): auxiliary classifier used during training
    """
    pass


class DeepLabHead(nn.Sequential):
    def __init__(self, in_channels, num_classes):
        super(DeepLabHead, self).__init__(
            ASPP(in_channels, [12, 24, 36]),
            nn.Conv2d(256, 256, 3, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, num_classes, 1)
        )


class ASPPConv(nn.Sequential):
    def __init__(self, in_channels, out_channels, dilation):
        modules = [
            nn.Conv2d(in_channels, out_channels, 3, padding=dilation, dilation=dilation, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        ]
        super(ASPPConv, self).__init__(*modules)


class ASPPPooling(nn.Sequential):
    def __init__(self, in_channels, out_channels):
        super(ASPPPooling, self).__init__(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU())

    def forward(self, x):
        size = x.shape[-2:]
      #  print(size)
        x = super(ASPPPooling, self).forward(x)
        return F.interpolate(x, size=size, mode='bilinear', align_corners=False)


class ASPP(nn.Module):
    def __init__(self, in_channels, atrous_rates):
        super(ASPP, self).__init__()
        out_channels = 256
        modules = []
        modules.append(nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()))

        rate1, rate2, rate3 = tuple(atrous_rates)
        modules.append(ASPPConv(in_channels, out_channels, rate1))
        modules.append(ASPPConv(in_channels, out_channels, rate2))
        modules.append(ASPPConv(in_channels, out_channels, rate3))
        modules.append(ASPPPooling(in_channels, out_channels))

        self.convs = nn.ModuleList(modules)

        self.project = nn.Sequential(
            nn.Conv2d(5 * out_channels, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Dropout(0.5))

    def forward(self, x):
        res = []
        for conv in self.convs:
            res.append(conv(x))
        res = torch.cat(res, dim=1)
        return self.project(res)

In [3]:
import torch
import torch.nn as nn


__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152', 'resnext50_32x4d', 'resnext101_32x8d',
           'wide_resnet50_2', 'wide_resnet101_2']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
    'resnext50_32x4d': 'https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth',
    'resnext101_32x8d': 'https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth',
    'wide_resnet50_2': 'https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth',
    'wide_resnet101_2': 'https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth',
}


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(1, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x


def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model


def resnet101(pretrained=False, progress=True, **kwargs):
    r"""ResNet-101 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>'_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
#     return _resnet('resnet101', Bottleneck, [3, 4, 23, 3], pretrained, progress, **kwargs)
    return _resnet('resnet101', Bottleneck, [3, 4, 9, 3], pretrained, progress, **kwargs)


In [4]:
from collections import OrderedDict

import torch
from torch import nn


class IntermediateLayerGetter(nn.ModuleDict):
    """
    Module wrapper that returns intermediate layers from a model
    It has a strong assumption that the modules have been registered
    into the model in the same order as they are used.
    This means that one should **not** reuse the same nn.Module
    twice in the forward if you want this to work.
    Additionally, it is only able to query submodules that are directly
    assigned to the model. So if `model` is passed, `model.feature1` can
    be returned, but not `model.feature1.layer2`.
    Arguments:
        model (nn.Module): model on which we will extract the features
        return_layers (Dict[name, new_name]): a dict containing the names
            of the modules for which the activations will be returned as
            the key of the dict, and the value of the dict is the name
            of the returned activation (which the user can specify).
    Examples::
        >>> m = torchvision.models.resnet18(pretrained=True)
        >>> # extract layer1 and layer3, giving as names `feat1` and feat2`
        >>> new_m = torchvision.models._utils.IntermediateLayerGetter(m,
        >>>     {'layer1': 'feat1', 'layer3': 'feat2'})
        >>> out = new_m(torch.rand(1, 3, 224, 224))
        >>> print([(k, v.shape) for k, v in out.items()])
        >>>     [('feat1', torch.Size([1, 64, 56, 56])),
        >>>      ('feat2', torch.Size([1, 256, 14, 14]))]
    """
    def __init__(self, model, return_layers):
        if not set(return_layers).issubset([name for name, _ in model.named_children()]):
            raise ValueError("return_layers are not present in model")

        orig_return_layers = return_layers
        return_layers = {k: v for k, v in return_layers.items()}
        layers = OrderedDict()
        for name, module in model.named_children():
            layers[name] = module
            if name in return_layers:
                del return_layers[name]
            if not return_layers:
                break

        super(IntermediateLayerGetter, self).__init__(layers)
        self.return_layers = orig_return_layers
        #self.softmax = nn.Softmax(dim=2)

    def forward(self, x):
        out = OrderedDict()
        for name, module in self.named_children():
            x = module(x)
            if name in self.return_layers:
                out_name = self.return_layers[name]
                out[out_name] = x#self.softmax(x)
        return out

In [5]:



__all__ = ['fcn_resnet50', 'fcn_resnet101', 'deeplabv3_resnet50', 'deeplabv3_resnet101']


model_urls = {
    'fcn_resnet50_coco': None,
    'fcn_resnet101_coco': 'https://download.pytorch.org/models/fcn_resnet101_coco-7ecb50ca.pth',
    'deeplabv3_resnet50_coco': None,
    'deeplabv3_resnet101_coco': 'https://download.pytorch.org/models/deeplabv3_resnet101_coco-586e9e4e.pth',
}


def _segm_resnet(name, backbone_name, num_classes, aux, pretrained_backbone=False):
    backbone = resnet101(
        pretrained=pretrained_backbone,
        replace_stride_with_dilation=[False, True, True])

    return_layers = {'layer4': 'out'}
    if aux:
        return_layers['layer3'] = 'aux'
    backbone = IntermediateLayerGetter(backbone, return_layers=return_layers)

    aux_classifier = None
    if aux:
        inplanes = 1024
        aux_classifier = FCNHead(inplanes, num_classes)

    model_map = {
        'deeplabv3': (DeepLabHead, DeepLabV3),
    }
    inplanes = 2048
    classifier = model_map[name][0](inplanes, num_classes)
    base_model = model_map[name][1]

    model = base_model(backbone, classifier, aux_classifier)
    return model


def _load_model(arch_type, backbone, pretrained, progress, num_classes, aux_loss, **kwargs):
    if pretrained:
        aux_loss = True
    model = _segm_resnet(arch_type, backbone, num_classes, aux_loss, **kwargs)
    if pretrained:
        arch = arch_type + '_' + backbone + '_coco'
        model_url = model_urls[arch]
        if model_url is None:
            raise NotImplementedError('pretrained {} is not supported as of now'.format(arch))
        else:
            state_dict = load_state_dict_from_url(model_url, progress=progress)
            model.load_state_dict(state_dict)
    return model

def deeplabv3_resnet101(pretrained=False, progress=True, 
                        num_classes=2, aux_loss=None, **kwargs):
    """Constructs a DeepLabV3 model with a ResNet-101 backbone.
    Args:
        pretrained (bool): If True, returns a model pre-trained on COCO train2017 which
            contains the same classes as Pascal VOC
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _load_model('deeplabv3', 'resnet101', pretrained, progress, num_classes, aux_loss, **kwargs)

In [6]:
model=deeplabv3_resnet101().cuda()

In [7]:

#criterion = nn.CrossEntropyLoss()
# model.load_state_dict(torch.load("deep3_(4_8).pkl"))
#criterion = nn.BCELoss()
learning_rate=0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
# import csv,os
# def get_csv():
#     class_path  =  'F:\\Ruiveen2\\Data\\Train_c1\\expert1\\'
#     class_path3  = 'F:\\Ruiveen2\\Data\\img\\'
#     blanket=set()
#     #############case3
# #     blanket.add(6)
# #     blanket.add(21)
# #     blanket.add(27)
# #################case1
#     blanket.add(8)
#     blanket.add(9)
#     blanket.add(2)
# ###############case2
# #     blanket.add(5)
# #     blanket.add(15)
# #     blanket.add(18)
# ###############case4
# #     blanket.add(7)
# #     blanket.add(29)
# #     blanket.add(34)

# #     if 2 in blanket:
# #         print("fdsfdsfds")
#     with open('my_test.csv','w',newline='')as f:
#         f_csv = csv.writer(f)
#         i=0
#         for img_name in os.listdir(class_path):
#             info=str.split(img_name,"_")
            
# #             print(img_name)
# #             break
#            # print(info[0])
#             if int(info[0]) in blanket:
#                 class_path2=class_path+img_name+"\\"
# #                 print(class_path2)
# #                 break
#                 for img_name2 in os.listdir(class_path2):
#                     img_name3=info[1]+"_"+img_name2
#                    # print(img_name3)
#                     img_path2= class_path2+ img_name2
#                     img_path = class_path3+info[1]+"\\"+ img_name2
# #                     print(img_path)
# #                     print(img_path2)
# #                     break
#                     temp=[2]
#                     temp.append(img_path)
#                     temp.append(img_name3)
#                     temp.append(img_path2)
#                     temp.append("")
#                     f_csv.writerow(temp)
#                     i=i+1
#                     if i%100==0:
#                         print(i)
# #                     break
#              #   break
# get_csv()


In [9]:
import csv,os
def get_csv():
    class_path  =  'F:\\Ruiveen2\\Data\\img\\'
    class_path3  = 'F:\\tq\\2020\\5\\23\\王铁桥-毕业论文-程序\\2.deeplab\\gen_t3\\'
    blanket=set()
    blanket2={}

    #############case3
    blanket.add(6)
    blanket.add(21)
    blanket.add(27)
#################case1
#     blanket.add(8)
#     blanket.add(9)
#     blanket.add(2)
###############case2
#     blanket.add(5)
#     blanket.add(15)
#     blanket.add(18)
###############case4
#     blanket.add(7)
#     blanket.add(29)
#     blanket.add(34)


#     for img_name in os.listdir(class_path):
#         info=str.split(img_name,"_")
#         blanket2[info[1]]=info[0]
#     for key in blanket2:
#         print(key+':'+blanket2[key])

#     return
    with open('case3_before_o.csv','w',newline='')as f:
        f_csv = csv.writer(f)
        i=0
        for img_name in os.listdir(class_path3):
            if img_name[0]!='_':
                continue
            info=str.split(img_name[1:],"_")
#             print(info)
            info3=info[0]
#             print(info3) #path
            info2=str.split(info[1],".")
            info2=info2[0]+".bmp"
#             print(info2) #file
            img_path2= class_path + info3+'\\'+info2
            img_path = class_path3+img_name
            temp=[2]
            temp.append(img_path2)
            img_name2=str.split(img_name[1:],".")
            img_name2=img_name2[0]+'.bmp'
            temp.append(img_name2)
            temp.append(img_path)
            temp.append("")
            f_csv.writerow(temp)
#             break
            i=i+1
            if i%100==0:
                print(i)
#             break

get_csv()


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


In [10]:
# def load_data2(label_path):
#     labels_list=open(label_path, 'r')
#     X,Y,Z=[],[],[]
#     i=1
#     c=0
#     d=0
#     for line in labels_list:
#         info=str.split(line,",") 
#         X.append(np.expand_dims(imageio.imread(info[3]),axis=2))#orginal image
#         a=imageio.imread(info[3])
#         b=np.max(a)
#         if b!=0:
#             b=1
#         if b:
#             d+=1
#         else:
#             c+=1
#         Y.append(2)
#         Z.append(str(info[2]))
#         #break
# #         print(X[0].shape)
# #         print(X[0].shape)
#         i+=1
#         if i%100==0:
#             print(i)
#     print(c,d)
#     return list(zip(X,Y,Z))

In [11]:
#keep the X contain the original image 
def load_data(label_path):
    labels_list=open(label_path, 'r')
    X,Y,Z,W=[],[],[],[]
    i=1
    c=0
    d=0
    for line in labels_list:
        info=str.split(line,",") 
        if i==1:
            print(info[1])
            print(info[3])
        X.append(np.expand_dims(imageio.imread(info[1]),axis=2))#orginal image
#         a=imageio.imread(info[3])
        W.append(np.expand_dims(imageio.imread(info[3]),axis=2))
#         b=np.max(a)
#         if b!=0:
#             b=1
#         if b:
#             d+=1
#         else:
#             c+=1
        Y.append(2)
        Z.append(str(info[2]))
        #break
#         print(X[0].shape)
#         print(X[0].shape)
        i+=1
        if i%100==0:
            print(i)
#     print(c,d)
    return list(zip(X,Y,Z,W))

In [12]:
import numpy as np
import torch
from torchvision.models import resnet101
import torch.nn as nn
import os
import cv2
import random
import torch.utils.model_zoo as model_zoo
from copy import deepcopy

from matplotlib import cm
import scipy.misc as misc
import matplotlib
import imageio
# data=load_data("/media/students/000FF23E00059C2D/tq/2019/8/20/gen_test2.csv")
# data=load_data("case4_before_o.csv")
data=load_data("case3_before_o.csv")

F:\Ruiveen2\Data\img\10072\1.bmp
F:\tq\2020\5\23\王铁桥-毕业论文-程序\2.deeplab\gen_t3\_10072_1.bmp
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


In [13]:
#model.load_state_dict(torch.load("deep4.pkl"))
mmpp=87685435454 # 1194110
model.load_state_dict(torch.load("deep3_o.pkl"))

<All keys matched successfully>

In [ ]:
import random #训练
batch_size=2
num_epochs=100
criterion = nn.CrossEntropyLoss()#torch.tensor([1,512]).float().cuda())

m=nn.Softmax(dim=1)
for epoch in range(num_epochs):
    count=0
    random.shuffle(data)
    X, Y,_,W= zip(*data)
    X=np.array(X).transpose((0,3,1,2))
    W=np.array(W).transpose((0,3,1,2))
    #print(W.shape)
    mm=0
    for i in range(0,X.shape[0],batch_size):
        if i+batch_size>=X.shape[0]: break
        batch=torch.from_numpy(X[i:i+batch_size]).cuda().float()
        outputs=model(batch)
        outputs['out']=m(outputs['out'])
      #  print(outputs)
        labels=torch.from_numpy(W[i:i+batch_size]).cuda().long()
        labels=torch.clamp(labels,0,1)
        labels=labels.view(labels.shape[0],labels.shape[2],labels.shape[3])
#             print(labels.shape)
#             print(torch.max(labels))
#             print(torch.min(labels))
#             print(outputs['out'].shape)
        loss = criterion(outputs['out'], labels)
        o_,pred=torch.max(outputs['out'],1)
        mm+=(pred!=labels).sum()
        #print(loss.item())
        #break
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if count%100==0:
            print('Epoch [{}/{}], step[{}/{}],Loss: {:.20f}'.format(epoch + 1, num_epochs, count,X.shape[0]//batch_size,loss.item()))
            print(mm)
        count += 1
    if mm<mmpp:
        mmpp=mm
        torch.save(model.state_dict(),"deep5_o.pkl")
#         learning_rate=min(0.02,learning_rate+0.0001)
#         optimizer = torch.optim.Ada m(model.parameters(), lr=learning_rate)
    else:
        model.load_state_dict(torch.load("deep5_o.pkl"))
#         learning_rate=0.0001
#         optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    print(mm)
    print(mmpp)

Epoch [1/100], step[0/1117],Loss: 1.24892139434814453125
tensor(984380, device='cuda:0')
